# V2 Model Server (SKLearn)

# Convert to function object

### Document and save (as a template)

In [1]:
import mlrun
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

fn = mlrun.code_to_function(
    items["name"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
    filename=items["spec"]["filename"],
    kind=items["spec"]["kind"],
    image=items["spec"]["image"],
)

fn.spec.default_class = "ClassifierModel"
# print(fn.to_yaml())
fn.export("v2_model_server.yaml")

> 2021-02-17 14:37:04,835 [info] function spec saved to path: v2_model_server.yaml


### Configure and add model(s)

In [2]:
models_path = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"
mlrun.mlconf.dbpath = mlrun.mlconf.dbpath or "http://mlrun-api:8080"
fn.add_model("mymodel", model_path=models_path)
# fn.verbose = True

# Example models locally (using a server emulator)

In [3]:
# create an emulator (mock server) from the function configuration)
from v2_model_server import ClassifierModel

server = fn.to_mock_server()

> 2021-02-17 14:37:05,875 [info] model mymodel was loaded
> 2021-02-17 14:37:05,875 [info] Loaded ['mymodel']


### Example against the iris dataset 

In [4]:
from sklearn.datasets import load_iris

iris = load_iris()
x = iris["data"].tolist()

In [5]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [6]:
fn.apply(mlrun.mount_v3io())

In [7]:
fn.deploy()

> 2021-02-17 14:37:05,951 [info] Starting remote function deploy
2021-02-17 14:37:06  (info) Deploying function
2021-02-17 14:37:06  (info) Building
2021-02-17 14:37:06  (info) Staging files and preparing base images
2021-02-17 14:37:06  (info) Building processor image
2021-02-17 14:37:09  (info) Build complete
2021-02-17 14:37:15  (info) Function deploy complete
> 2021-02-17 14:37:15,812 [info] function deployed, address=default-tenant.app.function-marketpace.iguazio-cd0.com:32440


'http://default-tenant.app.function-marketpace.iguazio-cd0.com:32440'

##  Example server

In [8]:
my_data = """{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}"""
fn.invoke("/v2/models/mymodel/infer", my_data)

{'id': 'e0e35a60-0bb1-476f-bdea-c6abce6d98be',
 'model_name': 'mymodel',
 'outputs': [0, 2]}